# Telecom Churn Group Project

This notebook is split into following section

<b>* Common Function</b>
    * This section we have all the common functions which are used in the entire notebook
<b> * Basic Data Analysis and Null Value imputation </b>
    * Drop Columns with more than 50% NULL Values
    * Handle columns with less number of NULL Values
    * Drop columns having not informative information
    * Create Dummy Variables for the categorical variables.
    * Filter and get high valued customer information
    * Derived Columns based on basic column analysis
    * Create a new columns which will tell about the Churn/No-Churn customer and Drop 9th month related columns
<b> * EDA </b>
    * EDA for Month 6 and 7 Together
    * EDA for Month 8
    * Derived Columns by combining 6th, 7th and 8th columns 
    * How the features are varied from Good Period to Decision period
    * What is the average variation from 6th+7th Month to 8th Month
    * EDA for the derived columns
<b> * Data Modeling </b>
    * Data Normalization 
    * Basic Logistic Regression Fit to Check the accuracy
    * As churn count is less, basic logist can be done with GridSearch and K-Fold
    * Using PCA dimentionality reduction can be done
    * By taking the PCA data logistic regression can be done again with GridSearch and K-Fold to check for prediction.
    * Using Ridge Regression, It can be found the important features impacting the churn.
    * Using Tree Model, Also Important Features can be derived.
    * If during EDA, any variable relation found with multinomial relation, then SVM Kernel can be used for prediction.

<b> * Final Model Selection </b>
    * Final Model for Prediction
    * Final Model for important Feature selection 
<b> * Summary </b>
    * Project Analysis and Summary